## 1. Imports packages

In [186]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime
import scipy.stats as stats
from sklearn.feature_selection import mutual_info_classif

print("Loaded Libraries")

Loaded Libraries


#### Set some options in Jupyter and for Matplotlib, Pandas and Seaborn.

In [187]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
#
plt.rcParams['figure.figsize'] = (3, 3)
plt.style.use('seaborn-pastel')
plt.rcParams['figure.dpi'] = 150      
#
c0 = sns.color_palette()[0]
c1 = sns.color_palette()[1]
c2 = sns.color_palette()[2]
#
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])
#

## 2. Load Data

We will use the cleaned data in EDA step

In [311]:
df_cleaned = pd.read_csv('../data/house_price_cleaned_data.csv')
df_cleaned.shape


print('The data have ', df_cleaned.shape[0], ' rows and ', df_cleaned.shape[1], ' columns\n')
print('column names: \n')
print('\n'.join(list(df_cleaned.columns)))

The data have  1459  rows and  59  columns

column names: 

MSSubClass
MSZoning
LotArea
Street
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
YearBuilt
YearRemodAdd
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
ExterQual
ExterCond
Foundation
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
Heating
HeatingQC
CentralAir
Electrical
2ndFlrSF
LowQualFinSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
KitchenQual
Functional
Fireplaces
GarageCars
PavedDrive
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
MiscVal
MoSold
YrSold
SaleType
SaleCondition
SalePrice


In [273]:
df_cleaned.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,Functional,Fireplaces,GarageCars,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,706,0,150,856,GasA,Ex,Y,SBrkr,854,0,1710,1,0,2,1,3,1,Gd,Typ,0,2,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,TA,TA,CBlock,978,0,284,1262,GasA,Ex,Y,SBrkr,0,0,1262,0,1,2,0,3,1,TA,Typ,1,2,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,486,0,434,920,GasA,Ex,Y,SBrkr,866,0,1786,1,0,2,1,3,1,Gd,Typ,1,2,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,TA,TA,BrkTil,216,0,540,756,GasA,Gd,Y,SBrkr,756,0,1717,1,0,1,0,3,1,Gd,Typ,1,3,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,655,0,490,1145,GasA,Ex,Y,SBrkr,1053,0,2198,1,0,2,1,4,1,Gd,Typ,1,3,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


#### Changing numeric variables into factors: MSSubClass
Recorded numeric but should actually be categorical

In [312]:
df_cleaned.replace({'MSSubClass': {20: '1-STORY 1946 & NEWER ALL STYLES',
        30: '1-STORY 1945 & OLDER',
        40: '1-STORY W/FINISHED ATTIC ALL AGES',
        45: '1-1/2 STORY - UNFINISHED ALL AGES',
        50: '1-1/2 STORY FINISHED ALL AGES',
        60: '2-STORY 1946 & NEWER',
        70: '2-STORY 1945 & OLDER',
        75: '2-1/2 STORY ALL AGES',
        80: 'SPLIT OR MULTI-LEVEL',
        85: 'SPLIT FOYER',
        90: 'DUPLEX - ALL STYLES AND AGES',
        120: '1-STORY PUD (Planned Unit Development) - 1946 & NEWER',
        150: '1-1/2 STORY PUD - ALL AGES',
        160: '2-STORY PUD - 1946 & NEWER',
        180: 'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER',
        190: '2 FAMILY CONVERSION - ALL STYLES AND AGES'}}, inplace=True)

In [275]:
df_cleaned.dtypes

MSSubClass       object
MSZoning         object
LotArea           int64
Street           object
LotShape         object
LandContour      object
Utilities        object
LotConfig        object
LandSlope        object
Neighborhood     object
Condition1       object
Condition2       object
BldgType         object
HouseStyle       object
OverallQual       int64
OverallCond       int64
YearBuilt         int64
YearRemodAdd      int64
RoofStyle        object
RoofMatl         object
Exterior1st      object
Exterior2nd      object
ExterQual        object
ExterCond        object
Foundation       object
BsmtFinSF1        int64
BsmtFinSF2        int64
BsmtUnfSF         int64
TotalBsmtSF       int64
Heating          object
HeatingQC        object
CentralAir       object
Electrical       object
2ndFlrSF          int64
LowQualFinSF      int64
GrLivArea         int64
BsmtFullBath      int64
BsmtHalfBath      int64
FullBath          int64
HalfBath          int64
BedroomAbvGr      int64
KitchenAbvGr    

In [276]:
df_cleaned.select_dtypes(include=['object']).columns

Index(['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object')

## 3. Determine the importance of categorical/numeric features vs target variable

Since we have 59 features, not all of them are highly correclted with SalePrice and not all of them provides useful information so we can use Anova & Kruskal-Wallis to determine the importance of categorical features on which feature to drop. 

#### 3.1 Use Anova to rank the categorical features vs our target variable SalePrice

In [277]:
# Separate the feature and target variable
target = df_cleaned['SalePrice']

anova_results = {}

for feature in df_cleaned.select_dtypes(include=['object']):
    # Perform ANOVA test for the current feature
    f_statistic, p_value = f_oneway(*(target[df_cleaned[feature] == category] for category in df_cleaned[feature].unique()))
    # Store the ANOVA results for the current feature in the dictionary
    anova_results[feature] = {'f_statistic': f_statistic, 'p_value': p_value}

# Sort the ANOVA results by the F-statistic in descending order
sorted_results = sorted(anova_results.items(), key=lambda x: x[1]['f_statistic'], reverse=True)

# Print the sorted ANOVA results
for feature, results in sorted_results:
    print('{}: F-statistic: {:.2f}, p-value: {:.2f}'.format(feature, results['f_statistic'], results['p_value']))

ExterQual: F-statistic: 443.12, p-value: 0.00
KitchenQual: F-statistic: 407.86, p-value: 0.00
Foundation: F-statistic: 100.37, p-value: 0.00
CentralAir: F-statistic: 98.27, p-value: 0.00
HeatingQC: F-statistic: 88.33, p-value: 0.00
Neighborhood: F-statistic: 71.91, p-value: 0.00
SaleCondition: F-statistic: 45.54, p-value: 0.00
MSZoning: F-statistic: 43.83, p-value: 0.00
PavedDrive: F-statistic: 42.01, p-value: 0.00
LotShape: F-statistic: 40.10, p-value: 0.00
MSSubClass: F-statistic: 33.71, p-value: 0.00
SaleType: F-statistic: 28.84, p-value: 0.00
Electrical: F-statistic: 23.07, p-value: 0.00
HouseStyle: F-statistic: 19.58, p-value: 0.00
Exterior1st: F-statistic: 18.63, p-value: 0.00
RoofStyle: F-statistic: 17.79, p-value: 0.00
Exterior2nd: F-statistic: 17.52, p-value: 0.00
BldgType: F-statistic: 13.01, p-value: 0.00
LandContour: F-statistic: 12.84, p-value: 0.00
ExterCond: F-statistic: 8.80, p-value: 0.00
LotConfig: F-statistic: 7.80, p-value: 0.00
RoofMatl: F-statistic: 6.72, p-value:

#### 3.2 Use Kruskal-Wallis test  to rank the categorical features vs our target variable SalePrice

In [278]:
# Select the categorical features and the target variable
cat_features = df_cleaned.select_dtypes(include=['object']).columns
target_var = 'SalePrice'

# Create an empty list to store the Kruskal-Wallis statistics and p-values for each feature
kw_stats = []
kw_pvals = []

# Loop through each categorical feature and perform the Kruskal-Wallis test with the target variable
for feature in cat_features:
    # Extract the unique categories and their corresponding values for the feature and target variable
    categories = df_cleaned[feature].unique()
    data = [df_cleaned.loc[df_cleaned[feature] == cat, target_var] for cat in categories]
    # Perform the Kruskal-Wallis test
    kw_stat, kw_pval = stats.kruskal(*data)
    # Append the results to the lists
    kw_stats.append(kw_stat)
    kw_pvals.append(kw_pval)

# Create a DataFrame to store the results
result_df = pd.DataFrame({'Feature': cat_features, 'Kruskal-Wallis': kw_stats, 'p-value': kw_pvals})

# Sort the DataFrame by p-value in ascending order
result_df = result_df.sort_values(by='p-value')

# Print the resulting DataFrame
print(result_df)

          Feature  Kruskal-Wallis        p-value
8    Neighborhood      868.588222  6.516987e-168
17      ExterQual      683.242943  9.026969e-148
24    KitchenQual      661.229406  5.353408e-143
0      MSSubClass      531.127944  2.315267e-104
19     Foundation      488.600192  2.305310e-103
21      HeatingQC      356.647307   6.436635e-76
1        MSZoning      269.871059   3.400769e-57
15    Exterior1st      303.957339   1.767007e-56
16    Exterior2nd      284.157152   1.085295e-51
12     HouseStyle      185.251313   1.514233e-36
28  SaleCondition      168.248296   1.725091e-34
22     CentralAir      143.142147   5.472066e-33
3        LotShape      150.933135   1.657554e-32
27       SaleType      156.314457   9.424908e-30
23     Electrical      131.423013   1.932218e-27
26     PavedDrive      115.785846   7.201501e-26
11       BldgType       69.739558   2.576180e-14
9      Condition1       74.336664   6.696776e-13
18      ExterCond       46.583093   1.862366e-09
4     LandContour   

Based on the tests above, we can confidentlly drop the below features as they don't indicate strong evidence in favor of an association between the categorical feature and the target variable: 
'Utilities', 'LandSlope', 'Condition2', 'Street', 'Heating', 'Functional', 'RoofMatl'

In [313]:
df_cleaned.drop(columns = ['Utilities', 'LandSlope', 'Condition2', 'Street', 'LandContour','Heating', 'Functional', 'RoofMatl']
             , inplace=True)

In [314]:
df_cleaned.shape

(1459, 51)

#### 3.4 Use Pearson correlation coefficient to measure the linear association between numeric features and  target variable 

In [281]:
# Select the numeric features and the target variable
num_features = df_cleaned.select_dtypes(exclude=['object']).columns
target_var = 'SalePrice'

# Calculate the Pearson correlation coefficient between each numeric feature and the target variable
corr_results = {}
for feature in num_features:
    corr_coef = df_cleaned[feature].corr(df_cleaned[target_var])
    corr_results[feature] = corr_coef

# Create a DataFrame to store the results
result_df = pd.DataFrame.from_dict(corr_results, orient='index', columns=['Correlation Coefficient'])

# Sort the DataFrame by correlation coefficient in descending order
result_df = result_df.sort_values(by='Correlation Coefficient', ascending=False)

# Print the resulting DataFrame
print(result_df)

               Correlation Coefficient
SalePrice                     1.000000
OverallQual                   0.791069
GrLivArea                     0.708618
GarageCars                    0.640473
TotalBsmtSF                   0.613905
FullBath                      0.560881
YearBuilt                     0.523273
YearRemodAdd                  0.507430
Fireplaces                    0.466968
BsmtFinSF1                    0.386436
WoodDeckSF                    0.324422
2ndFlrSF                      0.319464
OpenPorchSF                   0.315831
HalfBath                      0.284400
LotArea                       0.263837
BsmtFullBath                  0.227082
BsmtUnfSF                     0.214446
BedroomAbvGr                  0.168235
ScreenPorch                   0.111419
PoolArea                      0.092397
MoSold                        0.046380
3SsnPorch                     0.044571
BsmtFinSF2                   -0.011412
BsmtHalfBath                 -0.016873
MiscVal                  

From the Pearson correlation coefficient test, we can drop the feature with negative correlation to SalePrice

In [315]:
df_cleaned.drop(columns = ['BsmtFinSF2', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 
                   'YrSold','OverallCond','EnclosedPorch','KitchenAbvGr']
             , inplace=True)

In [394]:
df_pre = df_cleaned.copy()
print('Now we are have', df_pre.shape[1], 'features.')

Now we are have 43 features.


In [395]:
df_pre.head()

,MSSubClass,MSZoning,LotArea,LotShape,LotConfig,Neighborhood,Condition1,BldgType,HouseStyle,OverallQual,YearBuilt,YearRemodAdd,RoofStyle,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,HeatingQC,CentralAir,Electrical,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenQual,Fireplaces,GarageCars,PavedDrive,WoodDeckSF,OpenPorchSF,3SsnPorch,ScreenPorch,PoolArea,MoSold,SaleType,SaleCondition,SalePrice
0,2-STORY 1946 & NEWER,RL,8450,Reg,Inside,CollgCr,Norm,1Fam,2Story,7,2003,2003,Gable,VinylSd,VinylSd,Gd,TA,PConc,706,150,856,Ex,Y,SBrkr,854,1710,1,2,1,3,Gd,0,2,Y,0,61,0,0,0,2,WD,Normal,208500
1,1-STORY 1946 & NEWER ALL STYLES,RL,9600,Reg,FR2,Veenker,Feedr,1Fam,1Story,6,1976,1976,Gable,MetalSd,MetalSd,TA,TA,CBlock,978,284,1262,Ex,Y,SBrkr,0,1262,0,2,0,3,TA,1,2,Y,298,0,0,0,0,5,WD,Normal,181500
2,2-STORY 1946 & NEWER,RL,11250,IR1,Inside,CollgCr,Norm,1Fam,2Story,7,2001,2002,Gable,VinylSd,VinylSd,Gd,TA,PConc,486,434,920,Ex,Y,SBrkr,866,1786,1,2,1,3,Gd,1,2,Y,0,42,0,0,0,9,WD,Normal,223500
3,2-STORY 1945 & OLDER,RL,9550,IR1,Corner,Crawfor,Norm,1Fam,2Story,7,1915,1970,Gable,Wd Sdng,Wd Shng,TA,TA,BrkTil,216,540,756,Gd,Y,SBrkr,756,1717,1,1,0,3,Gd,1,3,Y,0,35,0,0,0,2,WD,Abnorml,140000
4,2-STORY 1946 & NEWER,RL,14260,IR1,FR2,NoRidge,Norm,1Fam,2Story,8,2000,2000,Gable,VinylSd,VinylSd,Gd,TA,PConc,655,490,1145,Ex,Y,SBrkr,1053,2198,1,2,1,4,Gd,1,3,Y,192,84,0,0,0,12,WD,Normal,250000


## 4.Transformation of categorical features

In [383]:
dfo = df_pre.select_dtypes(include=['object']) # select object type columns
df = pd.concat([df_pre.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1)
    
df.head()

,LotArea,OverallQual,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageCars,WoodDeckSF,OpenPorchSF,3SsnPorch,ScreenPorch,PoolArea,MoSold,SalePrice,MSSubClass_1-1/2 STORY - UNFINISHED ALL AGES,MSSubClass_1-1/2 STORY FINISHED ALL AGES,MSSubClass_1-STORY 1945 & OLDER,MSSubClass_1-STORY 1946 & NEWER ALL STYLES,MSSubClass_1-STORY PUD (Planned Unit Development) - 1946 & NEWER,MSSubClass_1-STORY W/FINISHED ATTIC ALL AGES,MSSubClass_2 FAMILY CONVERSION - ALL STYLES AND AGES,MSSubClass_2-1/2 STORY ALL AGES,MSSubClass_2-STORY 1945 & OLDER,MSSubClass_2-STORY 1946 & NEWER,MSSubClass_2-STORY PUD - 1946 & NEWER,MSSubClass_DUPLEX - ALL STYLES AND AGES,MSSubClass_PUD - MULTILEVEL - INCL SPLIT LEV/FOYER,MSSubClass_SPLIT FOYER,MSSubClass_SPLIT OR MULTI-LEVEL,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,8450,7,2003,2003,706,150,856,854,1710,1,2,1,3,0,2,0,61,0,0,0,2,208500,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,9600,6,1976,1976,978,284,1262,0,1262,0,2,0,3,1,2,298,0,0,0,0,5,181500,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0

In [384]:
df_pre.shape

(1459, 43)

## 5.Standardize the magnitude of numeric features using a scaler 

In [385]:
# Import StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_subset = df_pre[['LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1',
       'BsmtUnfSF', 'TotalBsmtSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'Fireplaces', 'GarageCars',
       'WoodDeckSF', 'OpenPorchSF', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MoSold']]

# Subset the DataFrame you want to scale 
df_subset = df_pre.select_dtypes(exclude=['object'])
names = df_subset.columns

# Apply the scaler to df_subset
df_scaled = scaler.fit_transform(df_subset)
scaled_df = pd.DataFrame(df_scaled, columns=names) 

df = df.drop(df_subset, axis=1)
df = pd.concat([df, scaled_df], axis=1)
df.head()
print(df.shape)

(1459, 184)


In [389]:
df.head()

,MSSubClass_1-1/2 STORY - UNFINISHED ALL AGES,MSSubClass_1-1/2 STORY FINISHED ALL AGES,MSSubClass_1-STORY 1945 & OLDER,MSSubClass_1-STORY 1946 & NEWER ALL STYLES,MSSubClass_1-STORY PUD (Planned Unit Development) - 1946 & NEWER,MSSubClass_1-STORY W/FINISHED ATTIC ALL AGES,MSSubClass_2 FAMILY CONVERSION - ALL STYLES AND AGES,MSSubClass_2-1/2 STORY ALL AGES,MSSubClass_2-STORY 1945 & OLDER,MSSubClass_2-STORY 1946 & NEWER,MSSubClass_2-STORY PUD - 1946 & NEWER,MSSubClass_DUPLEX - ALL STYLES AND AGES,MSSubClass_PUD - MULTILEVEL - INCL SPLIT LEV/FOYER,MSSubClass_SPLIT FOYER,MSSubClass_SPLIT OR MULTI-LEVEL,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,LotArea,OverallQual,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,Fireplaces,GarageCars,WoodDeckSF,OpenPorchSF,3SsnPorch,ScreenPorch,PoolArea,MoSold,SalePrice
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,-0.207125,0.650852,1.051899,0.879449,0.574748,-0.944607,-0.460570,1.162093,0.370055,1.107124,0.790180,1.228641,0.163837,-0.951848,0.311842,-0.751887,0.215983,-0.11638,-0.270308,-0.068715,-1.599030,0.347042
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## 6. Split into testing and training datasets 

In [390]:
from sklearn.model_selection import train_test_split

# dont forget to define your X and y
X = df.drop(columns = ['SalePrice'])
print("X: ", type(X), X.shape)

y = df['SalePrice']
print("y: ", type(y), y.shape)

X:  <class 'pandas.core.frame.DataFrame'> (1459, 183)
y:  <class 'pandas.core.series.Series'> (1459,)


In [391]:
X_train, X_test, y_train, y_test=train_test_split(X, y, 
                                                  test_size = 0.25,
                                                  random_state = 246)

In [392]:
# We now want to check the shape of the X train, y_train, X_test and y_test to make sure the proportions are right. 
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1094, 183) (1094,)
(365, 183) (365,)


## 7. Save preprocessed data to a new csv file

In [396]:
datapath = '../data'
save_file(df, 'house_price_preprocessed_data.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../data/house_price_preprocessed_data.csv"
